In [1]:
import pandas as pd
import re
# import spacy
import string
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import seaborn as sb
import plotly.express as px
import warnings

warnings.filterwarnings('ignore')
sb.set_context('notebook')

In [2]:
# all_emigrants_low = pd.read_csv('all_emigrants_low.csv').drop('Unnamed: 0', axis=1)
# all_emigrants_medium = pd.read_csv('all_emigrants_medium.csv').drop('Unnamed: 0', axis=1)
all_emigrants_high = pd.read_csv('all_emigrants_high.csv').drop('Unnamed: 0', axis=1)

In [3]:
all_emigrants_high

country_from     1980     1985      1990      1995      2000      2005  \
0     Afghanistan      0.0      0.0    1124.0    2239.0    3854.0    4553.0   
1         Albania    198.0    174.0     189.0     237.0     466.0     549.0   
2         Algeria      0.0      0.0     327.0     400.0     551.0     645.0   
3       Argentina   2079.0   2794.0    3958.0    4674.0    5208.0    6228.0   
4         Armenia    114.0    108.0     329.0     364.0     420.0     503.0   
...           ...      ...      ...       ...       ...       ...       ...   
1260    Venezuela   8096.0  14624.0   19811.0   29466.0   46043.0   78620.0   
1261      Vietnam  32560.0  82847.0  137341.0  179714.0  260005.0  385364.0   
1262        Yemen    416.0    741.0     965.0    1612.0    2750.0    4474.0   
1263       Zambia    384.0    860.0    1332.0     739.0     930.0    1352.0   
1264     Zimbabwe   1264.0   2123.0    2684.0    3586.0    5243.0    9246.0   

          2010     country_to      total  
0       5333.0      Australia    17103.0  
1        641.0      Australia     2454.0  
2        749.0      Australia     2672.0  
3       7361.0      Australia    32302.0  
4        596.0      Australia     2434.0  
...        ...            ...        ...  
1260   95187.0  United States   291847.0  
1261  445534.0  United States  1523365.0  
1262    5519.0  United States    16477.0  
1263    1544.0  United States     7141.0  
1264   10579.0  United States    34725.0  

[1265 rows x 10 columns]

In [5]:
new_years = set(pd.Series(np.arange(1980, 2011, 1)).map(lambda x: str(x))) - set(['1980', '1985', '1990', '1995', '2000', '2005', '2010'])
new_years = list(new_years)

for col in new_years:
    # all_emigrants_low[col] = np.nan
    # all_emigrants_medium[col] = np.nan
    all_emigrants_high[col] = np.nan

cols = list(pd.Series(np.arange(1980, 2011, 1)).map(lambda x: str(x)).values)
cols.insert(0, 'country_from')
cols.extend(['country_to', 'total'])


In [6]:
# all_emigrants_low = all_emigrants_low[cols]
# all_emigrants_low.iloc[:, 1:-2] = all_emigrants_low.iloc[:, 1:-2].interpolate(axis=1) 

# all_emigrants_medium = all_emigrants_medium[cols]
# all_emigrants_medium.iloc[:, 1:-2] = all_emigrants_medium.iloc[:, 1:-2].interpolate(axis=1)

all_emigrants_high = all_emigrants_high[cols]
all_emigrants_high.iloc[:, 1:-2] = all_emigrants_high.iloc[:, 1:-2].interpolate(axis=1)
all_emigrants_high

country_from     1980     1981     1982     1983     1984     1985  \
0     Afghanistan      0.0      0.0      0.0      0.0      0.0      0.0   
1         Albania    198.0    193.2    188.4    183.6    178.8    174.0   
2         Algeria      0.0      0.0      0.0      0.0      0.0      0.0   
3       Argentina   2079.0   2222.0   2365.0   2508.0   2651.0   2794.0   
4         Armenia    114.0    112.8    111.6    110.4    109.2    108.0   
...           ...      ...      ...      ...      ...      ...      ...   
1260    Venezuela   8096.0   9401.6  10707.2  12012.8  13318.4  14624.0   
1261      Vietnam  32560.0  42617.4  52674.8  62732.2  72789.6  82847.0   
1262        Yemen    416.0    481.0    546.0    611.0    676.0    741.0   
1263       Zambia    384.0    479.2    574.4    669.6    764.8    860.0   
1264     Zimbabwe   1264.0   1435.8   1607.6   1779.4   1951.2   2123.0   

         1986      1987      1988  ...      2003      2004      2005  \
0       224.8     449.6     674.4  ...    4273.4    4413.2    4553.0   
1       177.0     180.0     183.0  ...     515.8     532.4     549.0   
2        65.4     130.8     196.2  ...     607.4     626.2     645.0   
3      3026.8    3259.6    3492.4  ...    5820.0    6024.0    6228.0   
4       152.2     196.4     240.6  ...     469.8     486.4     503.0   
...       ...       ...       ...  ...       ...       ...       ...   
1260  15661.4   16698.8   17736.2  ...   65589.2   72104.6   78620.0   
1261  93745.8  104644.6  115543.4  ...  335220.4  360292.2  385364.0   
1262    785.8     830.6     875.4  ...    3784.4    4129.2    4474.0   
1263    954.4    1048.8    1143.2  ...    1183.2    1267.6    1352.0   
1264   2235.2    2347.4    2459.6  ...    7644.8    8445.4    9246.0   

          2006      2007      2008      2009      2010     country_to  \
0       4709.0    4865.0    5021.0    5177.0    5333.0      Australia   
1        567.4     585.8     604.2     622.6     641.0      Australia   
2        665.8     686.6     707.4     728.2     749.0      Australia   
3       6454.6    6681.2    6907.8    7134.4    7361.0      Australia   
4        521.6     540.2     558.8     577.4     596.0      Australia   
...        ...       ...       ...       ...       ...            ...   
1260   81933.4   85246.8   88560.2   91873.6   95187.0  United States   
1261  397398.0  409432.0  421466.0  433500.0  445534.0  United States   
1262    4683.0    4892.0    5101.0    5310.0    5519.0  United States   
1263    1390.4    1428.8    1467.2    1505.6    1544.0  United States   
1264    9512.6    9779.2   10045.8   10312.4   10579.0  United States   

          total  
0       17103.0  
1        2454.0  
2        2672.0  
3       32302.0  
4        2434.0  
...         ...  
1260   291847.0  
1261  1523365.0  
1262    16477.0  
1263     7141.0  
1264    34725.0  

[1265 rows x 34 columns]

In [7]:
# all_emigrants_low.to_csv('all_emigrants_low_interpolated.csv')
# all_emigrants_medium.to_csv('all_emigrants_medium_interpolated.csv')
all_emigrants_high.to_csv('all_emigrants_high_interpolated.csv')

In [102]:
data = pd.read_csv('Analysis_2021may.02075412.csv', skiprows=[1,2])

data.drop('Rank Indicators', axis=1, inplace=True)
data[' '][5:] = data[' .1'][5:]
data = data.drop(' .1', axis=1)
data = data.drop(4, axis=0)

In [130]:
data.iloc[:10, -150:-100]

Albania.4 Algeria.4  Angola.4 Argentina.4 Armenia.4 Australia.4 Austria.4  \
0             Weakness                                                         
1             Weakness  Strength    Strength              Weakness  Weakness   
2                       Weakness                                               
3             Weakness                                                         
5                                                                              
6                                                                   Strength   
7                       Strength                                    Strength   
8                                                         Strength             
9                                                                   Strength   
10                                  Weakness                        Strength   

   Azerbaijan.4 Bahrain.4 Bangladesh.4  ... El Salvador.4 Estonia.4  \
0                                       ...                           
1                             Strength  ...                           
2                             Weakness  ...                           
3                                       ...                           
5                                       ...                           
6                             Weakness  ...                           
7                             Weakness  ...                           
8                                       ...                           
9      Weakness  Weakness               ...      Strength  Strength   
10                                      ...                           

   Ethiopia.4    Fiji.4 Finland.4 France.4   Gabon.4  Gambia.4 Georgia.4  \
0                                                                          
1              Weakness  Weakness           Strength  Strength             
2                                                                          
3              Weakness                                                    
5                        Strength                                          
6                        Strength                                          
7                        Strength           Strength  Strength             
8                        Strength                                          
9                        Strength                                          
10                                                                         

   Germany.4  
0             
1             
2             
3             
5             
6             
7             
8             
9             
10            

[10 rows x 50 columns]

In [131]:
data

Albania  Algeria  Angola  \
0                    Global Innovation Index     93.0    138.0   135.0   
1                Innovation Efficiency Index    129.0    141.0    22.0   
2                           Innovation Input     77.0    112.0   140.0   
3                          Innovation Output    118.0    141.0   117.0   
5                               Institutions     73.0    118.0   136.0   
..                                       ...      ...      ...     ...   
112                        Online creativity     69.0    116.0   124.0   
113        Generic top-level domains (gTLDs)     87.0    121.0   128.0   
114  Country-code top-level domains (ccTLDs)     78.0    121.0   132.0   
115                  Wikipedia monthly edits     64.0    106.0   122.0   
116                 Video uploads on YouTube     41.0    104.0   118.0   

     Argentina  Armenia  Australia  Austria  Azerbaijan  Bahrain  ...  \
0         56.0     59.0       19.0     23.0       105.0     67.0  ...   
1         20.0     42.0      116.0     98.0       117.0    123.0  ...   
2         78.0     71.0       11.0     17.0        92.0     47.0  ...   
3         43.0     47.0       32.0     27.0       114.0     90.0  ...   
5        106.0     57.0       11.0     13.0        99.0     46.0  ...   
..         ...      ...        ...      ...         ...      ...  ...   
112       40.0     52.0        9.0     19.0        79.0     63.0  ...   
113       71.0     91.0        7.0     16.0        70.0     55.0  ...   
114       17.0     54.0       15.0      8.0        84.0     72.0  ...   
115       50.0     39.0       27.0     22.0        52.0     53.0  ...   
116       40.0     59.0       15.0     35.0        85.0     54.0  ...   

     United Kingdom.4  United States of America.4  Uruguay.4  Uzbekistan.4  \
0            Strength                                             Weakness   
1            Weakness                    Weakness                 Weakness   
2            Strength                    Strength                            
3            Strength                                             Weakness   
5                                                                            
..                ...                         ...        ...           ...   
112          Strength                               Strength                 
113                                      Strength                 Weakness   
114                                                                          
115                                                 Strength      Weakness   
116                                      Strength                 Weakness   

     Venezuela, Bolivarian Republic of.4  Viet Nam.4   Yemen.4  Zambia.4  \
0                                                     Weakness             
1                               Strength    Strength            Strength   
2                               Weakness              Weakness             
3                                                                          
5                               Weakness    Weakness                       
..                                   ...         ...       ...       ...   
112                                                                        
113                                                                        
114                             Strength                        Weakness   
115                                                                        
116                                                                        

     Zimbabwe.4  Unnamed: 752  
0      Weakness           NaN  
1      Strength           NaN  
2      Weakness           NaN  
3                         NaN  
5      Weakness           NaN  
..          ...           ...  
112                       NaN  
113                       NaN  
114                       NaN  
115    Weakness           NaN  
116                       NaN  

[116 rows x 751 columns]

In [ ]:
all_emigrants_high.sort_values(['total', 'country_from'], ascending=False).iloc[:27, 1:9].groupby('country_from').sum().reset_index().melt('country_from')